# Advanced tour of the Bayesian Optimization package

In [2]:
from bayes_opt import BayesianOptimization

## Suggest-Evaluate-Register Paradigm

Internally the `maximize` method is simply a wrapper around the methods `suggest`, `probe`, and `register`. If you need more control over your optimization loops the Suggest-Evaluate-Register paradigm should give you that extra flexibility.

For an example of running the `BayesianOptimization` in a distributed fashion (where the function being optimized is evaluated concurrently in different cores/machines/servers), checkout the `async_optimization.py` script in the examples folder.

In [3]:
# Lets start by definying our function, bounds, and instanciating an optimization object.
def black_box_function(x, y):
    return -x ** 2 - (y - 1) ** 2 + 1

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=1,
)

One extra ingredient we will need is an `UtilityFunction` instance. In case it is not clear why, take a look at the literature to understand better how this method works.

In [4]:
from bayes_opt import UtilityFunction

utility = UtilityFunction(kind="ucb", kappa=2.5, xi=0.0)

The `suggest` method of our optimizer can be called at any time. What you get back is a suggestion for the next parameter combination the optimizer wants to probe.

Notice that while the optimizer hasn't observed any points, the suggestions will be random. However they will stop being random and improve in quality the more points are observed.

In [5]:
next_point_to_probe = optimizer.suggest(utility)
print("Next point to probe is:", next_point_to_probe)

Next point to probe is: {'x': -0.331911981189704, 'y': 1.3219469606529488}


You are now free to evaluate you function at the suggested point however/whenever you like

In [6]:
target = black_box_function(**next_point_to_probe)
print("Found the target value to be:", target)

Found the target value to be: 0.7861845912690542


Last thing left to do is to tell the optimizer what target value was observed.

In [7]:
optimizer.register(
    params=next_point_to_probe,
    target=target,
)

And that's it. By repeating the steps above you recreate the internals of the `maximize` method. This should give you all the flexibility you need to log progress, hault execution, concurrent evaluations, etc.

In [9]:
for _ in range(5):
    next_point = optimizer.suggest(utility)
    target = black_box_function(**next_point)
    optimizer.register(params=next_point, target=target)
    
    print(target, next_point)
print(optimizer.max)

0.6991696847348962 {'x': 0.35745335256298433, 'y': 1.4160017019275122}
0.99784805556957 {'x': 0.04141972554410957, 'y': 1.0208890106582527}
0.9746135479061905 {'x': -0.15019450712870294, 'y': 0.9468204727157574}
0.9931443154738931 {'x': -0.0648518091817296, 'y': 1.0514774452742501}
0.9970772367740163 {'x': 0.004366164063039053, 'y': 0.9461140107508605}
{'target': 0.99784805556957, 'params': {'x': 0.04141972554410957, 'y': 1.0208890106582527}}


## Dealing with discrete parameters

**There is not principled way of dealing with discrete parameters using this package.**

Ok, now that we got that out of the way, how do you do it? You're bound to be in a situation where some of your function parameters may only take on discrete values. Unfortunately the nature of how bayesian optimization with gaussian processes is implement doesn't allow for an easy/intuitive way of dealing with discrete parameters. But that doesn't mean it is impossible.

In [85]:
def func_with_discrete_params(x, y, d):
    assert type(d) == int
    
    return  (x ** (1 / (d + y + 5))) / (1.1 ** (1.2 * x * d / y))

In [86]:
def function_to_be_optimized(x, y, w):
    d = int(w)
    return func_with_discrete_params(x, y, d)

In [87]:
optimizer = BayesianOptimization(
    f=function_to_be_optimized,
    pbounds={'x': (0, 10), 'y': (.1, 4), 'w': (0, 5)},
    verbose=2,
    random_state=1,
)

In [88]:
optimizer.maximize(gp_params={"alpha": 1e-3})

|   iter    |  target   |     w     |     x     |     y     |
-------------------------------------------------------------
|  1        |  9.924e-0 |  2.085    |  7.203    |  0.1004   |
|  2        |  0.7368   |  1.512    |  1.468    |  0.4601   |
|  3        |  1.205    |  0.9313   |  3.456    |  1.647    |
|  4        |  0.8194   |  2.694    |  4.192    |  2.772    |
|  5        |  0.01104  |  1.022    |  8.781    |  0.2068   |
|  6        |  0.0      |  0.0      |  0.0      |  4.0      |
|  7        |  1.227    |  0.0      |  6.317    |  4.0      |
|  8        |  0.2822   |  5.0      |  10.0     |  4.0      |
|  9        |  1.292    |  0.0      |  10.0     |  4.0      |
|  10       |  0.0      |  5.0      |  0.0      |  4.0      |
|  11       |  1.307    |  0.0      |  3.919    |  0.1      |
|  12       |  1.167    |  0.0      |  4.019    |  4.0      |
|  13       |  1.255    |  0.0      |  4.898    |  2.007    |
|  14       |  1.193    |  0.0      |  2.91     |  1.062    |
|  15   

KeyboardInterrupt: 

## Changing the utility function

## Tuning the underlying Gaussian Process

## Observers Continued